In [1]:
import pandas

from typing import List, Dict
from unidecode import unidecode

# loading dependencies
data = pandas.read_csv("estadao_noticias_eleicao.csv")

In [2]:
# cleaning null data
data = data.fillna('')

In [3]:
# joyning column of title and sub-title of artile with column of content.
data['articles'] = data['titulo']  + ' ' + data['subTitulo'] + ' ' + data['conteudo']

# lambda funcion to normalize text to lower case.
normalize = lambda text: unidecode(text.lower())

# lambda function to split text in tokens.
tokenize = lambda row: row.split()

# normalizing and tokenizing articles.
data['articles'] = data['articles'].apply(normalize)
data['tokens'] = data['articles'].apply(tokenize)

In [12]:
def create_index(matrix_of_tokens: List[str], doc_ids: List[int]) -> Dict[str, List[int]]:
    """Create a inverted index with all tokens and yours document Ids.
    :param matrix_of_tokens: matrix of article tokens lists.
    :param doc_ids: list of document ids of all articles.
    :returns: A inverted index with all tokens and yours document Ids.
    """
    index = {}
    for i in range(len(matrix_of_tokens)):
        for token in set(matrix_of_tokens[i]):
            if token in index.keys():
                index[token].append(doc_ids[i])
            else:
                index[token] = [doc_ids[i]]
    return index

In [13]:
# creating the inverted index
inverted_index = create_index(data['tokens'], data['idNoticia'])